ChatGPT first prompt :
- "i am building a RAG model using gpt-4o-mini as my generator. I want you to give me steps with code on how to incorporate R2AG into my RAG model. "


ChatGPT last prompt :
- "i am getting the following error here : 
TypeError: 'ChatCompletionMessage' object is not subscriptable"


Implementaton :

First, run the following command in the terminal to install langchain, transformers, faiss-cpu, and sentence-transformers for retrieval and embedding operations.


In [1]:
# pip install langchain transformers faiss-cpu sentence-transformers

In [2]:
import torch
import faiss
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

c:\Users\sehaj\OneDrive\Desktop\Capstone\FinChatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
class DocumentRetriever:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
        self.index = None  # FAISS Index

    def build_index(self, documents):
        embeddings = self.model.encode(documents, convert_to_numpy=True)
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)
        self.docs = documents  # Storing original documents

    def retrieve(self, query, top_k=3):
        query_embedding = self.model.encode([query], convert_to_numpy=True)
        distances, indices = self.index.search(query_embedding, top_k)
        return [self.docs[i] for i in indices[0]]


In [4]:
class R2Former(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(R2Former, self).__init__()
        self.projection = torch.nn.Linear(input_dim, hidden_dim)  # Projection layer
        self.self_attention = torch.nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=8)

    def forward(self, retrieval_embeddings):
        projected_embeddings = self.projection(retrieval_embeddings)
        attn_output, _ = self.self_attention(projected_embeddings, projected_embeddings, projected_embeddings)
        return attn_output


In [5]:
def transform_retrievals(retriever, query, r2former):
    retrieved_docs = retriever.retrieve(query)
    embeddings = retriever.model.encode(retrieved_docs, convert_to_tensor=True)
    embeddings = embeddings.unsqueeze(0)  # Reshaping for attention
    refined_embeddings = r2former(embeddings)
    return refined_embeddings, retrieved_docs


In [6]:
from openai import OpenAI

def generate_response(query, refined_embeddings, retrieved_docs):
    retrieval_context = " ".join(retrieved_docs)
    r2ag_prompt = f"Context: {retrieval_context}\n\nQuery: {query}\nAnswer:"
    
    client = OpenAI() 
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are an assistant."},
                  {"role": "user", "content": r2ag_prompt}]
    )
    return response.choices[0].message.content



In [7]:
retriever = DocumentRetriever()
retriever.build_index(["Document 1 text...", "Document 2 text...", "Document 3 text..."])  # Loading documents

r2former = R2Former(input_dim=384, hidden_dim=768)  # Matching dimensions of LLM

query = "What is R2AG?"
refined_embeddings, retrieved_docs = transform_retrievals(retriever, query, r2former)

response = generate_response(query, refined_embeddings, retrieved_docs)
print(response)


R2AG refers to "R2A Group," which is typically associated with a specific organization, group, or concept, but without additional context from the documents, it is difficult to specify its exact meaning or relevance. Please provide more details or context for a more accurate answer.
